In [50]:
import boto3, io, json
import pandas as pd

s3 = boto3.client("s3")
BUCKET = "rearcio-quest"

def get_bls_df_from_csv_from_s3(bucket, key):
    """Load BLS data from S3 CSV"""
    obj = s3.get_object(Bucket=bucket, Key=key)
    b = obj['Body'].read()
    try:
        df = pd.read_csv(io.BytesIO(b), sep='\t')
    except Exception:
        raise ValueError("Exception while reading BLS CSV data")
    return df

def clean_dataframe(df):
    """Clean DataFrame by stripping column names and string values"""
    # Clean column names
    df.columns = df.columns.str.strip()
    
    # Clean string columns
    for col in df.columns:
        if df[col].dtype == 'object':  # String columns
            df[col] = df[col].astype(str).str.strip()
    
    return df

def get_population_df_from_json_from_s3(bucket, key):
    """Load population data from S3 JSON"""
    obj = s3.get_object(Bucket=bucket, Key=key)
    pop_json = json.loads(obj['Body'].read())
    if isinstance(pop_json, dict):
        if "data" in pop_json and isinstance(pop_json["data"], list):
            df = pd.DataFrame(pop_json["data"])
        else:
            raise ValueError("Invalid population json data structure")
    else:
        raise ValueError("Invalid population json data structure")
    return df

pr_df = get_bls_df_from_csv_from_s3(BUCKET, "bls/data/pr.data.0.Current")
pr_df = clean_dataframe(pr_df)
print(f"BLS df\n{pr_df}\n")

pop_df = get_population_df_from_json_from_s3(BUCKET, "population/population_data.json")
pop_df = clean_dataframe(pop_df)

rename_map = {}
if 'Year' in pop_df.columns and 'year' not in pop_df.columns:
    rename_map['Year'] = 'year'
if 'Population' in pop_df.columns and 'population' not in pop_df.columns:
    rename_map['Population'] = 'population'
if 'Nation ID' in pop_df.columns and 'nation_id' not in pop_df.columns:
    rename_map['Nation ID'] = 'nation_id'
if 'Nation' in pop_df.columns and 'nation' not in pop_df.columns:
    rename_map['Nation'] = 'nation'
if rename_map:
    pop_df.rename(columns=rename_map, inplace=True)
print(f"Population df\n{pop_df}\n")

pop_df['year'] = pd.to_numeric(pop_df['year'], errors='coerce').astype('Int64')
pop_df['population'] = pd.to_numeric(pop_df['population'], errors='coerce')

# Population mean & std deviation for years 2013 to 2018 (inclusive)
pop_period = pop_df[(pop_df['year'] >= 2013) & (pop_df['year'] <= 2018)]
mean_pop = pop_period['population'].mean()
std_pop = round(pop_period['population'].std(ddof=0), 3)
print(f"Mean population (2013-2018): {mean_pop}")
print(f"Std dev population (2013-2018): {std_pop}\n")

# For every series_id find the best year (max sum of values across quarters)
# Ensure value is numeric
pr_df['value'] = pd.to_numeric(pr_df['value'], errors='coerce')
year_sums = pr_df.groupby(['series_id','year'], as_index=False)['value'].sum()
idx = year_sums.groupby('series_id')['value'].idxmax()
best_years = year_sums.loc[idx].reset_index(drop=True)
best_years.rename(columns={'value':'summed_value'}, inplace=True)
print(f"Best year for each series id\n {best_years}")

# Report for series_id = PRS30006032 and period = Q01, with population for that year
filter = (pr_df['series_id'] == 'PRS30006032') & (pr_df['period'] == 'Q01')
q1_df = pr_df[filter][['series_id','year','period','value']].copy()
q1_df['year'] = q1_df['year'].astype(int)
# Join with population
result = q1_df.merge(pop_df[['year','population']], on='year', how='left')
print(f"Value for series_id = PRS30006032 and period = Q01 and the population for that given year\n{result}")


BLS df
         series_id  year period    value footnote_codes
0      PRS30006011  1995    Q01    2.600            nan
1      PRS30006011  1995    Q02    2.100            nan
2      PRS30006011  1995    Q03    0.900            nan
3      PRS30006011  1995    Q04    0.100            nan
4      PRS30006011  1995    Q05    1.400            nan
...            ...   ...    ...      ...            ...
37234  PRS88003203  2024    Q03  116.593            nan
37235  PRS88003203  2024    Q04  116.682            nan
37236  PRS88003203  2024    Q05  116.686            nan
37237  PRS88003203  2025    Q01  118.271            nan
37238  PRS88003203  2025    Q02  118.482            nan

[37239 rows x 5 columns]

Population df
  nation_id         nation  year   population
0   01000US  United States  2013  316128839.0
1   01000US  United States  2014  318857056.0
2   01000US  United States  2015  321418821.0
3   01000US  United States  2016  323127515.0
4   01000US  United States  2017  325719178.0
5   